In [1]:
from google.cloud import bigquery
import google.auth
from google_auth_oauthlib import flow
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
import re
from datetime import date, timedelta
import os

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option("display.max_columns", None)

In [3]:
# the project that will pay for the queries
project_id='bolcom-pro-reco-analytics-fcc'
client = bigquery.Client(project_id)

/Users/bkersbergen/phd/etudelib/venv/lib/python3.9/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [17]:
def get_clicks(date_str):
    query = """
    with fct_sample as (
    SELECT
*
FROM
  `bolcom-pro-m2data-d35.views_analytical.measurements_analytical`
WHERE
  dateAmsterdam = "{date}"
  AND timeAmsterdam BETWEEN '20:00:00.000000' AND '21:00:00.000000'
    ),
    fct as (
select 
pii.session.sessionId as session_id,
REGEXP_EXTRACT(transport.pull.request.resource, '.*/p/[^/]+/(\\\d+).*') AS item_id,
UNIX_MILLIS(timestamp) as epoch_ms,
from `fct_sample`
where 
    event.container.details.page.name = "PDP"
    and event.container.details.containerType = 'PAGE'
    AND event.eventTargetType = 'CONTAINER'
    and event.container.event = 'RENDER'
    order by timestamp ASC
),
min_timestamp as (
    select 
    MIN(epoch_ms) as min_timestamp
    from `fct`    
),
session_map AS (
    SELECT 
        ROW_NUMBER() OVER() AS session_idx,
        session_id,
     FROM `fct`
     group by session_id
  ), 
item_map AS (
    SELECT 
        ROW_NUMBER() OVER() AS item_idx,
        item_id,
     FROM `fct`
     group by item_id
  )
SELECT 
  session_idx AS session_id,
  item_idx AS item_id,
  epoch_ms - min_timestamp.min_timestamp as epoch_ms
FROM 
  `fct`
JOIN 
  session_map ON `fct`.session_id = session_map.session_id
JOIN 
  item_map ON `fct`.item_id = item_map.item_id
CROSS JOIN 
  min_timestamp

      """.replace('{date}', str(date_str))
    df = client.query(query).to_dataframe()
    return df

In [18]:
the_date = date.today() - timedelta(days=1)
df = get_clicks(the_date)

In [19]:
df.to_csv('load.csv', index=False)